# Lab Questions Set2

### 1. Write a program to implement graph traversal (bfs, and dfs) using adjacency list.

In [1]:
class Queue:
    def __init__(self, capacity):
        self.queue = []
        self.capacity = capacity

    def enqueue(self, item):
        if len(self.queue) < self.capacity:
            self.queue.append(item)
        else:
            raise Exception("Queue Overflow")

    def dequeue(self):
        if not self.is_empty():
            return self.queue.pop(0)
        else:
            raise Exception("Queue Underflow")

    def is_empty(self):
        return len(self.queue) == 0

class Stack:
    def __init__(self, capacity):
        self.stack = []
        self.capacity = capacity

    def push(self, item):
        if len(self.stack) < self.capacity:
            self.stack.append(item)
        else:
            raise Exception("Stack Overflow")

    def pop(self):
        if not self.is_empty():
            return self.stack.pop()
        else:
            raise Exception("Stack Underflow")

    def is_empty(self):
        return len(self.stack) == 0

class Graph:
    def __init__(self):
        self.vertices = []
        self.adj_list = {}

    def add_vertex(self, v):
        if v not in self.adj_list:
            self.vertices.append(v)
            self.adj_list[v] = []

    def add_edge(self, u, v):
        self.add_vertex(u)
        self.add_vertex(v)
        if v not in self.adj_list[u]:
            self.adj_list[u].append(v)
        if u not in self.adj_list[v]:
            self.adj_list[v].append(u)

    def display(self):
        print("\n_____Adjacency List_____")
        for vertex, neighbors in self.adj_list.items():
            print(f"{vertex}: {neighbors}")

    def bfs(self, start):
        if start not in self.adj_list:
            return []

        n = len(self.vertices)
        STATUS = {v: 1 for v in self.vertices}
        traversal_order = []
        QUEUE = Queue(n)
        QUEUE.enqueue(start)
        STATUS[start] = 2

        while not QUEUE.is_empty():
            u = QUEUE.dequeue()
            traversal_order.append(u)
            STATUS[u] = 3
            for nbr in self.adj_list[u]:
                if STATUS[nbr] == 1:
                    QUEUE.enqueue(nbr)
                    STATUS[nbr] = 2
        return traversal_order

    def dfs(self, start):
        if start not in self.adj_list:
            return []

        n = len(self.vertices)
        STATUS = {v: 1 for v in self.vertices}
        traversal_order = []
        STACK = Stack(n)
        STACK.push(start)
        STATUS[start] = 2

        while not STACK.is_empty():
            u = STACK.pop()
            traversal_order.append(u)
            STATUS[u] = 3
            for nbr in reversed(self.adj_list[u]):
                if STATUS[nbr] == 1:
                    STACK.push(nbr)
                    STATUS[nbr] = 2
        return traversal_order

if __name__ == "__main__":
    g = Graph()
    m = int(input("Enter number of edges: "))
    print("Enter edges (u v) — undirected:")
    for _ in range(m):
        u, v = input().split()
        g.add_edge(u, v)

    g.display()
    start = input("Enter start vertex for traversal: ")
    print("\nBFS Traversal Order:")
    print(g.bfs(start))
    print("\nDFS Traversal Order:")
    print(g.dfs(start))

Enter number of edges: 5
Enter edges (u v) — undirected:
A B
A C
B D
C D
C E

_____Adjacency List_____
A: ['B', 'C']
B: ['A', 'D']
C: ['A', 'D', 'E']
D: ['B', 'C']
E: ['C']
Enter start vertex for traversal: A

BFS Traversal Order:
['A', 'B', 'C', 'D', 'E']

DFS Traversal Order:
['A', 'B', 'D', 'C', 'E']


### 2. Write a program to implement Minimum Spanning Tree algorithms (prims, and kruskal) using adjacency list.

In [3]:
import math

class MinHeap:
    def __init__(self):
        self.heap = []

    def is_empty(self):
        return len(self.heap) == 0

    def push(self, item):
        self.heap.append(item)
        self._heapify_up(len(self.heap) - 1)

    def pop(self):
        if self.is_empty():
            return None
        self._swap(0, len(self.heap) - 1)
        item = self.heap.pop()
        self._heapify_down(0)
        return item

    def _parent(self, i): return (i - 1) // 2
    def _left(self, i): return 2 * i + 1
    def _right(self, i): return 2 * i + 2

    def _swap(self, i, j):
        self.heap[i], self.heap[j] = self.heap[j], self.heap[i]

    def _heapify_up(self, i):
        while i > 0 and self.heap[i][0] < self.heap[self._parent(i)][0]:
            self._swap(i, self._parent(i))
            i = self._parent(i)

    def _heapify_down(self, i):
        smallest = i
        left = self._left(i)
        right = self._right(i)
        if left < len(self.heap) and self.heap[left][0] < self.heap[smallest][0]:
            smallest = left
        if right < len(self.heap) and self.heap[right][0] < self.heap[smallest][0]:
            smallest = right
        if smallest != i:
            self._swap(i, smallest)
            self._heapify_down(smallest)

class Graph:
    def __init__(self):
        self.vertices = []
        self.adj_list = {}

    def add_vertex(self, v):
        if v not in self.adj_list:
            self.vertices.append(v)
            self.adj_list[v] = []

    def add_edge(self, u, v, w):
        self.add_vertex(u)
        self.add_vertex(v)
        self.adj_list[u].append((v, w))
        self.adj_list[v].append((u, w))

    def display(self):
        print("\n_____Adjacency List_____")
        for vertex, neighbors in self.adj_list.items():
            print(f"{vertex}: {neighbors}")

    def prim(self):
        if not self.vertices:
            return []

        V = len(self.vertices)
        in_mst = {v: False for v in self.vertices}
        key = {v: math.inf for v in self.vertices}
        parent = {v: None for v in self.vertices}

        start = self.vertices[0]
        key[start] = 0

        for _ in range(V):
            u = None
            min_key = math.inf
            for v in self.vertices:
                if (not in_mst[v]) and key[v] < min_key:
                    min_key = key[v]
                    u = v
            if u is None:
                break
            in_mst[u] = True
            for (nbr, w) in self.adj_list[u]:
                if (not in_mst[nbr]) and w < key[nbr]:
                    key[nbr] = w
                    parent[nbr] = u

        T = []
        for v in self.vertices:
            if parent[v] is not None:
                T.append((parent[v], v, key[v]))
        print("Prim MST (edges u-v-weight):", T)
        return T

    def kruskal(self):
        edges = []
        seen = set()
        for u in self.adj_list:
            for v, w in self.adj_list[u]:
                if (v, u) not in seen:
                    edges.append((w, u, v))
                    seen.add((u, v))
        edges.sort(key=lambda x: x[0])

        parent = {v: v for v in self.vertices}
        rank = {v: 0 for v in self.vertices}

        def find(x):
            if parent[x] != x:
                parent[x] = find(parent[x])
            return parent[x]

        def union(x, y):
            rx = find(x)
            ry = find(y)
            if rx == ry: return False
            if rank[rx] < rank[ry]:
                parent[rx] = ry
            elif rank[ry] < rank[rx]:
                parent[ry] = rx
            else:
                parent[ry] = rx
                rank[rx] += 1
            return True

        T = []
        for w, u, v in edges:
            if union(u, v):
                T.append((u, v, w))
            if len(T) == len(self.vertices) - 1:
                break

        print("Kruskal MST (edges u-v-weight):", T)
        return T

if __name__ == "__main__":
    G = Graph()
    m = int(input("Enter number of weighted edges: "))
    print("Enter edges as: u v weight")
    for _ in range(m):
        u, v, w = input().split()
        G.add_edge(u, v, float(w))

    G.display()
    G.prim()
    G.kruskal()

Enter number of weighted edges: 6
Enter edges as: u v weight
A B 4
A C 2
B C 5
B D 10
C B 3
C D 4

_____Adjacency List_____
A: [('B', 4.0), ('C', 2.0)]
B: [('A', 4.0), ('C', 5.0), ('D', 10.0), ('C', 3.0)]
C: [('A', 2.0), ('B', 5.0), ('B', 3.0), ('D', 4.0)]
D: [('B', 10.0), ('C', 4.0)]
Prim MST (edges u-v-weight): [('C', 'B', 3.0), ('A', 'C', 2.0), ('C', 'D', 4.0)]
Kruskal MST (edges u-v-weight): [('A', 'C', 2.0), ('B', 'C', 3.0), ('C', 'D', 4.0)]


### 3. Write a program to implement Minimum Spanning Tree algorithms (prims, and kruskal) using minheap.

In [4]:
import math

class MinHeap:
    def __init__(self):
        self.heap = []

    def is_empty(self):
        return len(self.heap) == 0

    def push(self, item):
        self.heap.append(item)
        self._heapify_up(len(self.heap) - 1)

    def pop(self):
        if self.is_empty():
            return None
        self._swap(0, len(self.heap) - 1)
        item = self.heap.pop()
        self._heapify_down(0)
        return item

    def parent(self, i): return (i - 1) // 2
    def left(self, i): return 2 * i + 1
    def right(self, i): return 2 * i + 2

    def _swap(self, i, j):
        self.heap[i], self.heap[j] = self.heap[j], self.heap[i]

    def _heapify_up(self, i):
        while i > 0 and self.heap[i][0] < self.heap[self.parent(i)][0]:
            self._swap(i, self.parent(i))
            i = self.parent(i)

    def _heapify_down(self, i):
        smallest = i
        left = self.left(i)
        right = self.right(i)
        if left < len(self.heap) and self.heap[left][0] < self.heap[smallest][0]:
            smallest = left
        if right < len(self.heap) and self.heap[right][0] < self.heap[smallest][0]:
            smallest = right
        if smallest != i:
            self._swap(i, smallest)
            self._heapify_down(smallest)

class Graph:
    def __init__(self):
        self.vertices = []
        self.adj_list = {}

    def add_vertex(self, v):
        if v not in self.adj_list:
            self.vertices.append(v)
            self.adj_list[v] = []

    def add_edge(self, u, v, w):
        self.add_vertex(u)
        self.add_vertex(v)
        self.adj_list[u].append((v, w))
        self.adj_list[v].append((u, w))

    def display(self):
        print("\n_____Adjacency List_____")
        for vertex, neighbors in self.adj_list.items():
            print(f"{vertex}: {neighbors}")

    def prim_minheap(self):
        if not self.vertices: return []
        start = self.vertices[0]
        visited = {v: False for v in self.vertices}
        heap = MinHeap()
        for nbr, w in self.adj_list[start]:
            heap.push((w, start, nbr))
        visited[start] = True
        T = []
        while not heap.is_empty() and len(T) < len(self.vertices) - 1:
            w, u, v = heap.pop()
            if visited[v]:
                continue
            T.append((u, v, w))
            visited[v] = True
            for nbr, w2 in self.adj_list[v]:
                if not visited[nbr]:
                    heap.push((w2, v, nbr))
        print("Prim (MinHeap) MST (u-v-weight):", T)
        return T

    def kruskal_minheap(self):
        heap = MinHeap()
        seen = set()
        for u in self.adj_list:
            for v, w in self.adj_list[u]:
                if (v, u) not in seen:
                    heap.push((w, u, v))
                    seen.add((u, v))

        parent = {v: v for v in self.vertices}
        rank = {v: 0 for v in self.vertices}
        def find(x):
            if parent[x] != x:
                parent[x] = find(parent[x])
            return parent[x]
        def union(x, y):
            rx, ry = find(x), find(y)
            if rx == ry: return False
            if rank[rx] < rank[ry]:
                parent[rx] = ry
            elif rank[ry] < rank[rx]:
                parent[ry] = rx
            else:
                parent[ry] = rx
                rank[rx] += 1
            return True

        T = []
        while not heap.is_empty() and len(T) < len(self.vertices) - 1:
            w, u, v = heap.pop()
            if union(u, v):
                T.append((u, v, w))
        print("Kruskal (MinHeap) MST (u-v-weight):", T)
        return T

if __name__ == "__main__":
    G = Graph()
    m = int(input("Enter number of weighted edges: "))
    print("Enter edges as: u v weight")
    for _ in range(m):
        u, v, w = input().split()
        G.add_edge(u, v, float(w))

    G.display()
    G.prim_minheap()
    G.kruskal_minheap()

Enter number of weighted edges: 6
Enter edges as: u v weight
A B 4
A C 2
B C 5
B D 10
C D 3
C E 4

_____Adjacency List_____
A: [('B', 4.0), ('C', 2.0)]
B: [('A', 4.0), ('C', 5.0), ('D', 10.0)]
C: [('A', 2.0), ('B', 5.0), ('D', 3.0), ('E', 4.0)]
D: [('B', 10.0), ('C', 3.0)]
E: [('C', 4.0)]
Prim (MinHeap) MST (u-v-weight): [('A', 'C', 2.0), ('C', 'D', 3.0), ('C', 'E', 4.0), ('A', 'B', 4.0)]
Kruskal (MinHeap) MST (u-v-weight): [('A', 'C', 2.0), ('C', 'D', 3.0), ('A', 'B', 4.0), ('C', 'E', 4.0)]


### 4. Write a program to implement single source shortest path algorithms (Dijkstra's algorithm and  Bellman-Ford algorithm ) using adjacency list.

In [5]:
import math

class Graph:
    def __init__(self):
        self.vertices = []
        self.adj_list = {}

    def add_vertex(self, v):
        if v not in self.adj_list:
            self.vertices.append(v)
            self.adj_list[v] = []

    def add_edge(self, u, v, w):
        self.add_vertex(u)
        self.add_vertex(v)
        self.adj_list[u].append((v, w))
        self.adj_list[v].append((u, w))

    def display(self):
        print("\n_____Adjacency List_____")
        for vertex, neighbors in self.adj_list.items():
            print(f"{vertex}: {neighbors}")

    def dijkstra(self, src):
        V = len(self.vertices)
        dist = {v: math.inf for v in self.vertices}
        visited = {v: False for v in self.vertices}
        dist[src] = 0

        for _ in range(V):
            u = None
            min_dist = math.inf
            for v in self.vertices:
                if (not visited[v]) and dist[v] < min_dist:
                    min_dist = dist[v]
                    u = v
            if u is None:
                break
            visited[u] = True
            for (nbr, w) in self.adj_list[u]:
                if not visited[nbr] and dist[u] + w < dist[nbr]:
                    dist[nbr] = dist[u] + w
        print(f"Dijkstra distances from {src}:")
        print(dist)
        return dist

    def bellman_ford(self, src):
        V = len(self.vertices)
        dist = {v: math.inf for v in self.vertices}
        dist[src] = 0
        edges = []
        for u in self.adj_list:
            for v, w in self.adj_list[u]:
                edges.append((u, v, w))

        for _ in range(V - 1):
            for u, v, w in edges:
                if dist[u] != math.inf and dist[u] + w < dist[v]:
                    dist[v] = dist[u] + w

        for u, v, w in edges:
            if dist[u] != math.inf and dist[u] + w < dist[v]:
                print("Graph contains a negative-weight cycle!")
                return None

        print(f"Bellman-Ford distances from {src}:")
        print(dist)
        return dist

if __name__ == "__main__":
    G = Graph()
    m = int(input("Enter number of edges: "))
    print("Enter edges as: u v weight")
    for _ in range(m):
        u, v, w = input().split()
        G.add_edge(u, v, float(w))

    G.display()
    src = input("Enter source vertex: ")
    print()
    G.dijkstra(src)
    print()
    G.bellman_ford(src)

Enter number of edges: 6
Enter edges as: u v weight
A B 4
A C 2
B C 1
B D 2
C D 8
C E 2

_____Adjacency List_____
A: [('B', 4.0), ('C', 2.0)]
B: [('A', 4.0), ('C', 1.0), ('D', 2.0)]
C: [('A', 2.0), ('B', 1.0), ('D', 8.0), ('E', 2.0)]
D: [('B', 2.0), ('C', 8.0)]
E: [('C', 2.0)]
Enter source vertex: A

Dijkstra distances from A:
{'A': 0, 'B': 3.0, 'C': 2.0, 'D': 5.0, 'E': 4.0}

Bellman-Ford distances from A:
{'A': 0, 'B': 3.0, 'C': 2.0, 'D': 5.0, 'E': 4.0}


### 5. Write a program to implement single source shortest path algorithms (Dijkstra's algorithm) using minheap.

In [6]:
import math

class MinHeap:
    def __init__(self):
        self.heap = []

    def is_empty(self):
        return len(self.heap) == 0

    def push(self, item):
        self.heap.append(item)
        self._heapify_up(len(self.heap) - 1)

    def pop(self):
        if self.is_empty():
            return None
        self._swap(0, len(self.heap) - 1)
        item = self.heap.pop()
        self._heapify_down(0)
        return item

    def _parent(self, i): return (i - 1) // 2
    def _left(self, i): return 2 * i + 1
    def _right(self, i): return 2 * i + 2

    def _swap(self, i, j):
        self.heap[i], self.heap[j] = self.heap[j], self.heap[i]

    def _heapify_up(self, i):
        while i > 0 and self.heap[i][0] < self.heap[self._parent(i)][0]:
            self._swap(i, self._parent(i))
            i = self._parent(i)

    def _heapify_down(self, i):
        smallest = i
        left = self._left(i)
        right = self._right(i)
        if left < len(self.heap) and self.heap[left][0] < self.heap[smallest][0]:
            smallest = left
        if right < len(self.heap) and self.heap[right][0] < self.heap[smallest][0]:
            smallest = right
        if smallest != i:
            self._swap(i, smallest)
            self._heapify_down(smallest)

class Graph:
    def __init__(self):
        self.vertices = []
        self.adj_list = {}

    def add_vertex(self, v):
        if v not in self.adj_list:
            self.vertices.append(v)
            self.adj_list[v] = []

    def add_edge(self, u, v, w):
        self.add_vertex(u)
        self.add_vertex(v)
        self.adj_list[u].append((v, w))
        self.adj_list[v].append((u, w))

    def display(self):
        print("\n_____Adjacency List_____")
        for vertex, neighbors in self.adj_list.items():
            print(f"{vertex}: {neighbors}")

    def dijkstra_minheap(self, src):
        dist = {v: math.inf for v in self.vertices}
        dist[src] = 0
        visited = {v: False for v in self.vertices}
        heap = MinHeap()
        heap.push((0, src))

        while not heap.is_empty():
            d, u = heap.pop()
            if d > dist[u]:
                continue
            visited[u] = True
            for nbr, w in self.adj_list[u]:
                if dist[u] + w < dist[nbr]:
                    dist[nbr] = dist[u] + w
                    heap.push((dist[nbr], nbr))
        print(f"Dijkstra (MinHeap) distances from {src}:")
        print(dist)
        return dist

if __name__ == "__main__":
    G = Graph()
    m = int(input("Enter number of weighted edges: "))
    print("Enter edges as: u v weight")
    for _ in range(m):
        u, v, w = input().split()
        G.add_edge(u, v, float(w))

    G.display()
    src = input("Enter source vertex: ")
    G.dijkstra_minheap(src)

Enter number of weighted edges: 5
Enter edges as: u v weight
A B 4
A C 2
B D 7
C D 1
D E 3

_____Adjacency List_____
A: [('B', 4.0), ('C', 2.0)]
B: [('A', 4.0), ('D', 7.0)]
C: [('A', 2.0), ('D', 1.0)]
D: [('B', 7.0), ('C', 1.0), ('E', 3.0)]
E: [('D', 3.0)]
Enter source vertex: A
Dijkstra (MinHeap) distances from A:
{'A': 0, 'B': 4.0, 'C': 2.0, 'D': 3.0, 'E': 6.0}


### 6. Write a program to implement All-Pairs Shortest Path algorithms (Floyd Warshall) using adjacency list.

In [7]:

import math

class Graph:
    def __init__(self):
        self.vertices = []
        self.adj_list = {}

    def add_vertex(self, v):
        if v not in self.adj_list:
            self.vertices.append(v)
            self.adj_list[v] = []

    def add_edge(self, u, v, w):
        self.add_vertex(u)
        self.add_vertex(v)
        self.adj_list[u].append((v, w))
        self.adj_list[v].append((u, w))

    def display(self):
        print("\n_____Adjacency List_____")
        for vertex, neighbors in self.adj_list.items():
            print(f"{vertex}: {neighbors}")

    def to_matrix(self):
        n = len(self.vertices)
        idx = {v: i for i, v in enumerate(self.vertices)}
        mat = [[math.inf] * n for _ in range(n)]
        for i in range(n):
            mat[i][i] = 0
        for u in self.adj_list:
            for v, w in self.adj_list[u]:
                mat[idx[u]][idx[v]] = min(mat[idx[u]][idx[v]], w)
        return mat, idx

    def floyd_warshall(self):
        mat, idx = self.to_matrix()
        n = len(mat)
        # Floyd-Warshall
        dist = [row[:] for row in mat]
        for k in range(n):
            for i in range(n):
                for j in range(n):
                    if dist[i][k] + dist[k][j] < dist[i][j]:
                        dist[i][j] = dist[i][k] + dist[k][j]
        print("\nAll-Pairs Shortest Path Matrix (rows/cols = vertices order):")
        print(self.vertices)
        for i, row in enumerate(dist):
            print(self.vertices[i], ":", row)
        return dist

if __name__ == "__main__":
    G = Graph()
    m = int(input("Enter number of weighted edges: "))
    print("Enter edges as: u v weight")
    for _ in range(m):
        u, v, w = input().split()
        G.add_edge(u, v, float(w))

    G.display()
    G.floyd_warshall()

Enter number of weighted edges: 5
Enter edges as: u v weight
A B 4
A C 2
B C 5
B  D 10
C D 3

_____Adjacency List_____
A: [('B', 4.0), ('C', 2.0)]
B: [('A', 4.0), ('C', 5.0), ('D', 10.0)]
C: [('A', 2.0), ('B', 5.0), ('D', 3.0)]
D: [('B', 10.0), ('C', 3.0)]

All-Pairs Shortest Path Matrix (rows/cols = vertices order):
['A', 'B', 'C', 'D']
A : [0, 4.0, 2.0, 5.0]
B : [4.0, 0, 5.0, 8.0]
C : [2.0, 5.0, 0, 3.0]
D : [5.0, 8.0, 3.0, 0]
